In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils_groc import get_groc_api_key

groc_api_key = get_groc_api_key()
os.environ["GROQ_API_KEY"] = 'deepseek-r1-distill-llama-70b'

In [4]:
researcher = Agent(
    role="Digital Marketing Researcher",
    goal="Gather accurate, relevant, and engaging information on {topic} to support the creation of a high-quality blog post.",
    backstory="You are an expert in digital marketing research, specializing in identifying trends, data, and insights that resonate with the target audience. "
              "Your role is to collect information that is factual, up-to-date, and valuable to the audience. "
              "Your research will serve as the foundation for the Content Writer to create an engaging and informative blog post.",
    instructions=[
        "1. Identify the target audience and their preferences for the topic: {topic}.",
        "2. Conduct thorough research using credible sources such as industry reports, case studies, and reputable websites.",
        "3. Gather data, statistics, and trends that support the topic and provide value to the audience.",
        "4. Organize the research into clear sections (e.g., introduction, key points, examples, and conclusions).",
        "5. Ensure all information is accurate, up-to-date, and properly cited.",
        "6. Provide a summary of the research findings to the Content Writer for further development."],
    allow_delegation=False,
	verbose=True
)

LLM value is None


In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write a compelling, engaging, and well-structured blog post on {topic} based on the research provided.",
    backstory="You are a skilled content writer with expertise in crafting blog posts that resonate with the target audience. "
              "Your role is to transform the research findings into a clear, engaging, and informative article that aligns with the company's tone and style.",
    instructions=[
        "1. Review the research provided by the Researcher and identify the key points to include in the blog post.",
        "2. Write a captivating headline and introduction to grab the reader's attention.",
        "3. Structure the blog post into sections (e.g., introduction, main body, conclusion) for easy readability.",
        "4. Use clear, concise, and engaging language to communicate the information effectively.",
        "5. Incorporate relevant examples, statistics, and quotes from the research to support your points.",
        "6. Ensure the tone and style align with the company's brand guidelines.",
        "7. Optimize the content for SEO by including relevant keywords and meta descriptions.",
        "8. Submit the draft to the Editor for review and refinement."
    ],
    allow_delegation=False,
    verbose=True
)

LLM value is None


In [6]:
editor = Agent(
    role="Content Editor",
    goal="Review and refine the blog post to ensure it is polished, error-free, appropiate for the company's policy and ready for publication.",
    backstory="You are an experienced content editor with a keen eye for detail and a deep understanding of the company's brand voice. "
              "Your role is to ensure the blog post is clear campared to the {topic}, engaging, and free of errors before it is published.",
    instructions=[
        "1. Review the blog post draft for grammar, spelling, and punctuation errors.",
        "2. Ensure the content is clear, concise, and easy to understand.",
        "3. Verify that the tone and style align with the company's brand guidelines.",
        "4. Check that the blog post is well-structured and flows logically.",
        "5. Confirm that all facts, statistics, and quotes are accurate and properly cited.",
        "6. Provide constructive feedback to the Content Writer if revisions are needed.",
        "7. Finalize the blog post and prepare it for publication."
    ],
    allow_delegation=False,
    verbose=True
)

LLM value is None


# Tools

In [7]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool,  WebsiteSearchTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Tasks

In [8]:
plan = Task(
    description=(
        "1. Analyze the topic: {topic} and identify its relevance to the target audience.\n"
        "2. Research the latest trends, statistics, and news related to {topic}.\n"
        "3. Identify key players, influencers, or brands in the industry related to {topic}.\n"
        "4. Gather credible sources such as industry reports, case studies, and reputable websites.\n"
        "5. Highlight pain points, challenges, and opportunities related to {topic}.\n"
        "6. Organize the research into clear sections (e.g., trends, data, examples, and sources)."
    ),
    expected_output=(
        "A detailed research document containing:\n"
        "- Key trends and statistics related to {topic}.\n"
        "- Credible sources and references.\n"
        "- Insights into the target audience's pain points and interests.\n"
        "- A structured outline of the research findings."
    ),
    tools = [search_tool, scrape_tool],
    agent=researcher,
)

In [9]:
audience_analysis = Task(
    description=(
        "1. Define the target audience for the blog post on {topic}.\n"
        "2. Identify the audience's demographics, interests, and preferences.\n"
        "3. Analyze the audience's pain points and how {topic} can address them.\n"
        "4. Determine the tone and style that will resonate with the audience.\n"
        "5. Provide recommendations for engaging the audience effectively."
    ),
    expected_output=(
        "An audience analysis report containing:\n"
        "- Demographics and psychographics of the target audience.\n"
        "- Pain points and interests related to {topic}.\n"
        "- Recommendations for tone, style, and engagement strategies."
    ),
    tools = [search_tool, scrape_tool],
    agent=researcher,
)

In [10]:
# writer task

write = Task(
    description=(
        "1. Use the research document and audience analysis to create a blog draft.\n"
        "2. Write a compelling headline and introduction to grab the reader's attention.\n"
        "3. Structure the blog post into sections (e.g., introduction, main body, conclusion).\n"
        "4. Incorporate key points, examples, and statistics from the research.\n"
        "5. Use a tone and style that aligns with the target audience and brand guidelines.\n"
        "6. Include a clear call-to-action (CTA) to engage the audience.\n"
        "7. Optimize the content for SEO by including relevant keywords and meta descriptions."
    ),
    expected_output=(
        "A complete blog draft containing:\n"
        "- A captivating headline and introduction.\n"
        "- Well-structured sections with clear key points.\n"
        "- Examples, statistics, and quotes from the research.\n"
        "- A strong call-to-action (CTA).\n"
        "- SEO-optimized content with keywords and meta descriptions."
    ),
    tools = [search_tool, scrape_tool],
    agent=writer,
)

In [11]:
revise_blog = Task(
    description=(
        "1. Review the Editor's feedback on the blog draft.\n"
        "2. Make necessary revisions to improve clarity, tone, and structure.\n"
        "3. Ensure all facts, statistics, and quotes are accurate and properly cited.\n"
        "4. Double-check SEO optimization, including keywords and meta descriptions.\n"
        "5. Submit the revised draft to the Editor for final approval."
    ),
    expected_output=(
        "A revised blog draft that incorporates all feedback and is ready for final review."
    ),
    tools = [search_tool, scrape_tool],
    agent=writer,
)

In [12]:
# editot task

edit = Task(
    description=(
        "1. Review the blog draft for grammar, spelling, and punctuation errors.\n"
        "2. Ensure the content is clear, concise, and easy to understand.\n"
        "3. Verify that the tone and style align with the company's brand guidelines.\n"
        "4. Check that the blog post is well-structured and flows logically.\n"
        "5. Confirm that all facts, statistics, and quotes are accurate and properly cited.\n"
        "6. Provide constructive feedback to the Writer for revisions if needed."
    ),
    expected_output=(
        "A reviewed blog draft with:\n"
        "- Corrections for grammar, spelling, and punctuation.\n"
        "- Suggestions for improving clarity, tone, and structure.\n"
        "- Confirmation of factual accuracy and proper citations.\n"
        "- Feedback for the Writer if revisions are required."
    ),
    tools = [search_tool, scrape_tool],
    agent=editor,
)

In [13]:
finalize_blog = Task(
    description=(
        "1. Review the revised blog draft from the Writer.\n"
        "2. Ensure all feedback has been incorporated and the content is error-free.\n"
        "3. Confirm that the blog post is optimized for SEO and aligns with brand guidelines.\n"
        "4. Prepare the blog post for publication by formatting it appropriately.\n"
        "5. Submit the finalized blog post for publishing."
    ),
    expected_output=(
        "A finalized blog post that is:\n"
        "- Error-free and polished.\n"
        "- SEO-optimized and aligned with brand guidelines.\n"
        "- Ready for publication."
    ),
    tools = [search_tool, scrape_tool],
    agent=editor,
)

# Crew

In [ ]:
crew = Crew(
    agents=[researcher, writer, editor],
    tasks=[plan, audience_analysis, write, revise_blog, edit, finalize_blog],  # tasks oprates swquentially
    verbose=True,  # should be boolean
    memory = True
)

# Run Crew

In [15]:
from IPython.display import Markdown
topic = "Write a blog post about How Microsoft Learn Student Ambassador - Sri Lanka community helps to sharp students skills in Sri Lanka"
result = crew.kickoff(inputs={"topic": topic})
Markdown(result.raw)

# Agent: Digital Marketing Researcher
## Task: 1. Analyze the topic: Write a blog post about How Microsoft Learn Student Ambassador - Sri Lanka community helps to sharp students skills in Sri Lanka and identify its relevance to the target audience.
2. Research the latest trends, statistics, and news related to Write a blog post about How Microsoft Learn Student Ambassador - Sri Lanka community helps to sharp students skills in Sri Lanka.
3. Identify key players, influencers, or brands in the industry related to Write a blog post about How Microsoft Learn Student Ambassador - Sri Lanka community helps to sharp students skills in Sri Lanka.
4. Gather credible sources such as industry reports, case studies, and reputable websites.
5. Highlight pain points, challenges, and opportunities related to Write a blog post about How Microsoft Learn Student Ambassador - Sri Lanka community helps to sharp students skills in Sri Lanka.
6. Organize the research into clear sections (e.g., trends, data,

```
# Empowering Future Innovators: How the Microsoft Learn Student Ambassador Community is Shaping Skills in Sri Lanka

The digital landscape is rapidly evolving, and with it, the need for skilled professionals in technology continues to grow. In Sri Lanka, the Microsoft Learn Student Ambassador community stands out as a transformative platform, equipping students with essential skills that transcend their university education. This vibrant network empowers aspiring professionals to thrive in a competitive job market by enhancing their technical proficiency and soft skills. In this blog post, we will explore how this dynamic community fosters personal and professional growth among students in Sri Lanka.

## Bridging the Skills Gap

One of the most significant challenges faced by students today is the gap between theoretical knowledge learned in classrooms and the practical skills sought by employers. The Microsoft Learn Student Ambassador program addresses this concern head-on by offering hands-on workshops and resources tailored to current industry demands.

Participants engage in a variety of events, such as coding boot camps and cloud computing workshops, where they gain firsthand experience with cutting-edge technologies. For instance, students use Azure and AI tools in projects designed to solve real-world problems. According to recent surveys, **75% of participants reported feeling more confident in their technical abilities** after attending workshops hosted by the community (source: [Microsoft Learn](https://learn.microsoft.com/en-us/training/)). This direct application of skills not only prepares students for the workforce but also builds a strong portfolio that they can present to future employers.

## Building Connections and Networking

In the competitive tech landscape, effective networking can significantly impact career opportunities. The Microsoft Learn Student Ambassador program offers students the chance to establish connections with industry leaders through exclusive events and mentorship initiatives.

Networking opportunities allow students to expand their professional contacts and gain insights into the tech industry from seasoned professionals. Mentorship sessions featuring experts from Microsoft and other tech giants provide students with insider knowledge and practical tips that can elevate their career paths. As reported by a participant, “The mentorship I received was instrumental in securing my internship,” emphasizing the transformative potential of these connections. Furthermore, these relationships can lead to internships and job placements, effectively bridging the transition from education to employment.

## Fostering Leadership and Community Engagement

The Microsoft Learn Student Ambassador community focuses not just on technical skills but also underscores the importance of leadership and community service. Ambassadors are encouraged to take the lead on initiatives that benefit their peers and the broader community, such as organizing hackathons and coding clubs. This approach nurtures essential soft skills, including communication, teamwork, and project management.

A notable project led by the Sri Lankan ambassadors involved developing a mobile app that assists local farmers in optimizing their supply chains. Such initiatives showcase the technical acumen of students and reinforce their commitment to making a positive impact within their communities. By merging technical skills with community service, ambassadors are not only enhancing their resumes but also ensuring they are valued contributors to society.

## A Call to Action: Join the Movement!

For university students in Sri Lanka who are passionate about technology and looking to enhance their skill sets, the Microsoft Learn Student Ambassador program represents an invaluable opportunity. Whether you aspire to deepen your technical knowledge, expand your professional network, or develop leadership qualities, this community can serve as your springboard to success.

Are you ready to take your first step toward becoming a Microsoft Learn Student Ambassador? Visit the [Microsoft Learn Student Ambassadors](https://mvp.microsoft.com/en-US/studentambassadors/apply) website and apply today!

## Conclusion

The Microsoft Learn Student Ambassador community in Sri Lanka plays a pivotal role in shaping the skills of future innovators and leaders in technology. By bridging the skills gap, fostering connections, and promoting leadership, the program equips students to thrive in the ever-evolving digital world. Join the movement today and kickstart your journey toward a promising career in technology!

---

**Meta Description**: Discover how the Microsoft Learn Student Ambassador community in Sri Lanka empowers students to bridge the skills gap, connect with industry leaders, and develop essential leadership qualities. Join today and enhance your tech skills for a brighter future!
```

git init
git add README.md
git commit -m "first commit"
git branch -M main
git remote add origin __link__
git push -u origin main

In [16]:
'''notion agent/Include/ 
function calling'''

'notion agent/Include/ \nfunction calling'